In [1]:
!rocm-smi --showproductname



============================ ROCm System Management Interface ============================
====================================== Product Info ======================================
GPU[0]		: Card Series: 		AMD Instinct MI300X VF
GPU[0]		: Card Model: 		0x74b5
GPU[0]		: Card Vendor: 		Advanced Micro Devices, Inc. [AMD/ATI]
GPU[0]		: Card SKU: 		M3000100
GPU[0]		: Subsystem ID: 	0x74a1
GPU[0]		: Device Rev: 		0x00
GPU[0]		: Node ID: 		1
GPU[0]		: GUID: 		21947
GPU[0]		: GFX Version: 		gfx942
================================== End of ROCm SMI Log ===================================


In [2]:
!sudo apt update && sudo apt install git -y

Hit:1 https://repo.radeon.com/amdgpu/30.10_rc1/ubuntu noble InRelease
Hit:2 https://repo.radeon.com/graphics/7.0_rc1/ubuntu noble InRelease          
Hit:3 https://repo.radeon.com/rocm/apt/7.0_rc1 noble InRelease                 
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease               
Hit:5 http://archive.ubuntu.com/ubuntu noble InRelease                         
Hit:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu noble InRelease
Hit:8 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
22 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.43.0-1ubuntu7.3).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [3]:
%%capture
%pip install -U transformers
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes

In [28]:
!pip uninstall torch torchvision torchaudio -y 

Found existing installation: torch 2.4.1+rocm6.0
Uninstalling torch-2.4.1+rocm6.0:
  Successfully uninstalled torch-2.4.1+rocm6.0
Found existing installation: torchvision 0.19.1+rocm6.0
Uninstalling torchvision-0.19.1+rocm6.0:
  Successfully uninstalled torchvision-0.19.1+rocm6.0
Found existing installation: torchaudio 2.4.1+rocm6.0
Uninstalling torchaudio-2.4.1+rocm6.0:
  Successfully uninstalled torchaudio-2.4.1+rocm6.0


In [4]:
!pip install torch==2.4.1+rocm6.0 --index-url https://download.pytorch.org/whl/rocm6.0 --no-deps --force-reinstall --no-cache-dir
!pip install torchvision==0.19.1+rocm6.0 --index-url https://download.pytorch.org/whl/rocm6.0 --no-deps --force-reinstall --no-cache-dir  
!pip install torchaudio==2.4.1+rocm6.0 --index-url https://download.pytorch.org/whl/rocm6.0 --no-deps --force-reinstall --no-cache-dir

Looking in indexes: https://download.pytorch.org/whl/rocm6.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 GB 62.4 MB/s  0:00:37m0:00:0100:02m
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1+rocm6.0
    Uninstalling torch-2.4.1+rocm6.0:
      Successfully uninstalled torch-2.4.1+rocm6.0
Looking in indexes: https://download.pytorch.org/whl/rocm6.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 MB 123.3 MB/s  0:00:00eta 0:00:01
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.1+rocm6.0
    Uninstalling torchvision-0.19.1+rocm6.0:
      Successfully uninstalled torchvision-0.19.1+rocm6.0
Looking in indexes: https://download.pytorch.org/whl/rocm6.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.9 MB/s  0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.1+rocm6.0
    Uninstalling torchaudio-2.4.1+rocm6.0:
      Successfully uninstalled torchaudio-2.4.1+rocm6.

In [4]:

#downloaded the wrong torch lol

In [5]:
!pip install transformers datasets huggingface-hub scipy -qq

In [6]:
import os
import torch

print("Environment check:")
print(f"ROCM_PATH: {os.getenv('ROCM_PATH', 'Not set')}")
print(f"HIP_VISIBLE_DEVICES: {os.getenv('HIP_VISIBLE_DEVICES', 'Not set')}")

print(f"\nPyTorch version: {torch.__version__}")
print(f"ROCm available: {torch.cuda.is_available()}")
print(f"Device count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f"Current device: {device}")
    print(f"Device name: {torch.cuda.get_device_name(device)}")
    print(f"Device properties: {torch.cuda.get_device_properties(device)}")
    
    # Test tensor operations
    x = torch.randn(3, 3).cuda()
    y = torch.randn(3, 3).cuda()
    z = x + y
    print(f"Test computation successful: {z.shape}")

Environment check:
ROCM_PATH: Not set
HIP_VISIBLE_DEVICES: Not set

PyTorch version: 2.4.1+rocm6.0
ROCm available: True
Device count: 1
Current device: 0
Device name: AMD Instinct MI300X VF
Device properties: _CudaDeviceProperties(name='AMD Instinct MI300X VF', major=9, minor=4, gcnArchName='gfx942:sramecc+:xnack-', total_memory=196288MB, multi_processor_count=304)
Test computation successful: torch.Size([3, 3])


In [7]:
!pip install trl -qq
!pip install peft -qq

In [8]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:

from huggingface_hub import login
login(token=os.environ["HUGGING_FACE_TOKEN"])


In [11]:
!huggingface-cli whoami

⚠️  Warning: 'huggingface-cli whoami' is deprecated. Use 'hf auth whoami' instead.
Advik-7


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Loading the Model and Tokenizer (AMD ROCm variant)
model_dir = "microsoft/Phi-4-reasoning-plus"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# ROCm specific loading - explicit device placement instead of device_map="auto"
model = AutoModelForCausalLM.from_pretrained(
    model_dir, 
    torch_dtype=torch.bfloat16,  # ROCm supports bfloat16
    trust_remote_code=True
)

# Move to ROCm device (ROCm uses cuda syntax)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Check ROCm memory usage
print("ROCm device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
print("Memory allocated:", torch.cuda.memory_allocated(0) / 1024**3, "GB")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

ROCm device: AMD Instinct MI300X VF
Memory allocated: 27.383586883544922 GB


In [13]:
train_prompt_style = """
<|im_start|>system<|im_sep|>
Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<|im_end|>
<|im_start|>user<|im_sep|>
{}<|im_end|>
<|im_start|>assistant<|im_sep|>
<think>
{}
</think>
{}
"""

In [14]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["Open-ended Verifiable Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, complex_cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [16]:
from datasets import load_dataset

dataset = load_dataset(
    "TheFinAI/Fino1_Reasoning_Path_FinQA", split="train[0:1000]"
)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [17]:
inference_prompt_style = """
<|im_start|>system<|im_sep|>
Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<|im_end|>
<|im_start|>user<|im_sep|>
{}<|im_end|>
<|im_start|>assistant<|im_sep|>
<think>
{}
"""

In [ ]:
# Number of samples
print("Number of rows:", len(dataset))

# Inspect a random example
print(dataset[0])

In [21]:
question = dataset[20]['Open-ended Verifiable Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to(device)  # Use our ROCm device

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print("Before fine-tuning:")
print(response[0].split("<|im_start|>assistant<|im_sep|>")[1])

Before fine-tuning:
<think><|im_end|>We are asked: "what was the percentage increase in the employee contribution from 2002 to 2003". The context: "retirement plan: The company has a 401(k) plan covering substantially all employees who meet certain age and employment requirements. Under the plan, the company matching contribution for periods prior to June 30, 2004 was 35% (35%) up to a maximum 5% (5%) of a participant's contributions. Effective July 1, 2004, the plan was amended to increase the company match to 50% (50%) up to a maximum 6% (6%) of a participant's contributions. The company contributed approximately $533,000, $825,000 and $979,000 to the plan for the years ended December 31, 2004, 2003 and 2002, respectively."

Wait, the question: "what was the percentage increase in the employee contribution from 2002 to 2003?" But what exactly means employee contribution? It says "employee contribution" but in context, the provided numbers: For 2004, 2003, 2002, the company contribute

In [23]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

model = get_peft_model(model, peft_config)


/opt/venv/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/opt/venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [26]:

training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="adamw_torch",
    num_train_epochs=1,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="none",
    dataloader_pin_memory=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

trainer.train()

/opt/venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
100,1.445700
200,1.362600
300,1.281900
400,1.321600
500,1.342200


TrainOutput(global_step=500, training_loss=1.350809295654297, metrics={'train_runtime': 720.7619, 'train_samples_per_second': 1.387, 'train_steps_per_second': 0.694, 'total_flos': 8.592683944836096e+16, 'train_loss': 1.350809295654297, 'epoch': 1.0})

In [27]:

question = dataset[20]['Open-ended Verifiable Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to(device)

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print("After fine-tuning:")
print(response[0].split("<|im_start|>assistant<|im_sep|>")[1])

After fine-tuning:
<think><|im_end|>Alright, let's get started on figuring out the percentage increase in the employee contribution from 2002 to 2003. First off, we need to look at the contributions for each year. The data says that in 2002, the contributions were $979,000. Moving on to 2003, the contributions went up to $825,000.

Now, I should calculate the difference between these two years. That's easy enough; I just subtract the 2002 amount from the 2003 amount. So, $825,000 minus $979,000 gives us -$154,000. Oh, it looks like there's a decrease here, not an increase.

Hmm, let's pause here. The problem specifically asks about the percentage increase in the employee contribution. But according to the numbers we have, there's actually a decrease from 2002 to 2003. So, it's not about an increase at all.

Let's double-check those numbers to make sure I didn't miss anything. Back in 2002, the contributions were indeed $979,000, and in 2003, it's $825,000. So, we're seeing a reduction,

In [28]:

new_model_name = "Phi-4-Reasoning-Plus-FinQA-COT-ROCm"
model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)
print(f"Model saved as: {new_model_name}")


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

  2025-09-27T16:41:41.949092Z  WARN  Status Code: 504. Retrying..., request_id: ""
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:227



README.md: 0.00B [00:00, ?B/s]

Model saved as: Phi-4-Reasoning-Plus-FinQA-COT-ROCm


In [29]:

# ROCm memory cleanup
torch.cuda.empty_cache()
gc.collect()

127